# Building a Basic RAG

In [1]:
from dotenv import load_dotenv
import os
import tqdm

# Load environment variables from .env file
load_dotenv()

# Access the API keys
openai_api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
# from haystack import Document
# from haystack import Pipeline
# from haystack.document_stores.in_memory import InMemoryDocumentStore
# from haystack.components.embedders import OpenAITextEmbedder, OpenAIDocumentEmbedder
# from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

# document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")

# documents = [Document(content="My name is Wolfgang and I live in Berlin"),
#              Document(content="I saw a black horse running"),
#              Document(content="Germany has many big cities")]

# document_embedder = OpenAIDocumentEmbedder()
# documents_with_embeddings = document_embedder.run(documents)['documents']
# document_store.write_documents(documents_with_embeddings)

# query_pipeline = Pipeline()
# query_pipeline.add_component("text_embedder", OpenAITextEmbedder())
# query_pipeline.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store))


# query_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")

# query = "Who lives in Berlin?"

# result = query_pipeline.run({"text_embedder":{"text": query}})

# print(result['retriever']['documents'][0])

/Users/adamfletcher/Documents/GitHub/find-techspike-rag/.venv/lib/python3.12/site-packages/haystack/core/errors.py:34: DeprecationWarning: PipelineMaxLoops is deprecated and will be remove in version '2.7.0'; use PipelineMaxComponentRuns instead.
  warnings.warn(
/Users/adamfletcher/Documents/GitHub/find-techspike-rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.13it/s]


Document(id=62fad790ad2af927af9432c87330ed2ea5e31332cdec8e9d6235a5105ab0aaf5, content: 'My name is Wolfgang and I live in Berlin', score: 0.8843885516161921)


## Let's Add the LLM

In [2]:
from haystack.components.builders import PromptBuilder

template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)

/Users/adamfletcher/Documents/GitHub/find-techspike-rag/.venv/lib/python3.12/site-packages/haystack/core/errors.py:34: DeprecationWarning: PipelineMaxLoops is deprecated and will be remove in version '2.7.0'; use PipelineMaxComponentRuns instead.
  warnings.warn(
/Users/adamfletcher/Documents/GitHub/find-techspike-rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from haystack import Document
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import OpenAITextEmbedder, OpenAIDocumentEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.generators import OpenAIGenerator

document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")
generator = OpenAIGenerator(model="gpt-4o-mini")

documents = [Document(content="My name is Wolfgang and I live in Berlin"),
             Document(content="I saw a black horse running"),
             Document(content="Germany has many big cities")]

document_embedder = OpenAIDocumentEmbedder()
documents_with_embeddings = document_embedder.run(documents)['documents']
document_store.write_documents(documents_with_embeddings)

query_pipeline = Pipeline()
query_pipeline.add_component("text_embedder", OpenAITextEmbedder())
query_pipeline.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store))
query_pipeline.add_component("prompt_builder", prompt_builder)
query_pipeline.add_component("llm", generator)

query_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
query_pipeline.connect("retriever", "prompt_builder.documents")
query_pipeline.connect("prompt_builder", "llm")


Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  2.59it/s]


🚅 Components
  - text_embedder: OpenAITextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [8]:

query = "Who lives in Berlin?"

result = query_pipeline.run({"text_embedder":{"text": query}, "prompt_builder": {"question": query}})

# print(result['retriever']['documents'][0])
print(result['llm']['replies'][0])

Wolfgang lives in Berlin.


### Can I make a chat interface

In [4]:
from haystack import Document
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import OpenAITextEmbedder, OpenAIDocumentEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.generators.chat import OpenAIChatGenerator

document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")
generator = OpenAIChatGenerator(model="gpt-4o-mini")

documents = [Document(content="My name is Wolfgang and I live in Berlin"),
             Document(content="I saw a black horse running"),
             Document(content="Germany has many big cities")]

document_embedder = OpenAIDocumentEmbedder()
documents_with_embeddings = document_embedder.run(documents)['documents']
document_store.write_documents(documents_with_embeddings)

query_pipeline = Pipeline()
query_pipeline.add_component("text_embedder", OpenAITextEmbedder())
query_pipeline.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store))
# query_pipeline.add_component("prompt_builder", prompt_builder)
query_pipeline.add_component("llm", generator)

query_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
query_pipeline.connect("retriever", "llm")

Calculating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.17it/s]


PipelineConnectError: Cannot connect 'retriever' with 'llm': no matching connections available.
'retriever':
 - documents: List[Document]
'llm':
 - messages: List[ChatMessage] (available)
 - streaming_callback: Optional[Callable[]] (available)
 - generation_kwargs: Optional[Dict[str, Any]] (available)

In [18]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack.components.generators.chat import OpenAIChatGenerator

template = """
Answer the questions based on the given context.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}
Question: {{ question }}
Answer:
"""
rag_pipe = Pipeline()
rag_pipe.add_component("embedder", OpenAITextEmbedder())
rag_pipe.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store))
rag_pipe.add_component("prompt_builder", PromptBuilder(template=template))
rag_pipe.add_component("llm", OpenAIChatGenerator(model="gpt-4o-mini"))

rag_pipe.connect("embedder.embedding", "retriever.query_embedding")
rag_pipe.connect("retriever", "prompt_builder.documents")
rag_pipe.connect("prompt_builder", "llm")

PipelineConnectError: Cannot connect 'prompt_builder' with 'llm': no matching connections available.
'prompt_builder':
 - prompt: str
'llm':
 - messages: List[ChatMessage] (available)
 - streaming_callback: Optional[Callable[]] (available)
 - generation_kwargs: Optional[Dict[str, Any]] (available)

In [10]:
query = "Where does Mark live?"
rag_pipe.run({"embedder": {"text": query}, "prompt_builder": {"question": query}})

{'embedder': {'meta': {'model': 'text-embedding-ada-002',
   'usage': {'prompt_tokens': 5, 'total_tokens': 5}}},
 'llm': {'replies': ['The context does not provide any information about a person named Mark, so it is not possible to answer where he lives. The only person mentioned is Wolfgang, who lives in Berlin.'],
  'meta': [{'model': 'gpt-4o-mini-2024-07-18',
    'index': 0,
    'finish_reason': 'stop',
    'usage': {'completion_tokens': 36,
     'prompt_tokens': 54,
     'total_tokens': 90,
     'completion_tokens_details': CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0),
     'prompt_tokens_details': PromptTokensDetails(audio_tokens=0, cached_tokens=0)}}]}}

In [5]:
def rag_pipeline_func(query: str):
    result = rag_pipe.run({"embedder": {"text": query}, "prompt_builder": {"question": query}})

    return {"reply": result["llm"]["replies"][0]}

WEATHER_INFO = {
    "Berlin": {"weather": "mostly sunny", "temperature": 7, "unit": "celsius"},
    "Paris": {"weather": "mostly cloudy", "temperature": 8, "unit": "celsius"},
    "Rome": {"weather": "sunny", "temperature": 14, "unit": "celsius"},
    "Madrid": {"weather": "sunny", "temperature": 10, "unit": "celsius"},
    "London": {"weather": "cloudy", "temperature": 9, "unit": "celsius"},
}


def get_current_weather(location: str):
    if location in WEATHER_INFO:
        return WEATHER_INFO[location]

    # fallback data
    else:
        return {"weather": "sunny", "temperature": 21.8, "unit": "fahrenheit"}

available_functions = {"rag_pipeline_func": rag_pipeline_func, "get_current_weather": get_current_weather}


import gradio as gr
import json

from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import OpenAIChatGenerator

chat_generator = OpenAIChatGenerator(model="gpt-4o-mini")
response = None
messages = [
    ChatMessage.from_system(
        "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
    )
]


def chatbot_with_fc(message, history):
    messages.append(ChatMessage.from_user(message))
    response = chat_generator.run(messages=messages, generation_kwargs={"tools": tools})

    while True:
        # if OpenAI response is a tool call
        if response and response["replies"][0].meta["finish_reason"] == "tool_calls":
            function_calls = json.loads(response["replies"][0].content)
            print(response["replies"][0])
            for function_call in function_calls:
                ## Parse function calling information
                function_name = function_call["function"]["name"]
                function_args = json.loads(function_call["function"]["arguments"])

                ## Find the correspoding function and call it with the given arguments
                function_to_call = available_functions[function_name]
                function_response = function_to_call(**function_args)

                ## Append function response to the messages list using `ChatMessage.from_function`
                messages.append(ChatMessage.from_function(content=json.dumps(function_response), name=function_name))
                response = chat_generator.run(messages=messages, generation_kwargs={"tools": tools})

        # Regular Conversation
        else:
            messages.append(response["replies"][0])
            break
    return response["replies"][0].content


